## Quickstart: Compare runs, chose model and delpoy it to Rest API

In [10]:
## Librairies
import keras
import pandas as pd
import numpy as np
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [11]:
## load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-red.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [12]:
## Split the data into train, validation and tests sets
train,test = train_test_split(data,test_size=0.25,random_state=42)

In [13]:
train_x = train.drop(['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

## Test data
test_x = test.drop(['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

## splitting this train into train and validation
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)

In [14]:
## ANN Model
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## define model architecture
    mean = np.mean(train_x,axis=0)
    var = np.var(train_x,axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate = params['lr'],
        momentum = params['momentum']
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with MLFlow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)

        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log params and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## Log model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [15]:
def objective(params):
    ## MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result

In [16]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [17]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

 1/15 ━━━━━━━━━━━━━━━━━━━━ 6s 441ms/step - loss: 36.4575 - root_mean_squared_error: 6.0380
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 33.3582 - root_mean_squared_error: 5.7735 - val_loss: 24.7035 - val_root_mean_squared_error: 4.9703

Epoch 2/3                                            

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 25.1105 - root_mean_squared_error: 5.0110
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 23.3650 - root_mean_squared_error: 4.8323 - val_loss: 17.3504 - val_root_mean_squared_error: 4.1654

Epoch 3/3                                            

 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 18.9389 - root_mean_squared_error: 4.3519
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 16.4845 - root_mean_squared_error: 4.0583 - val_loss: 12.0794 - val_root_mean_squared_error: 3.4755

1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 12.1191 - root_mean_squared_error: 3.4812
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 11.9165 - root_mean_squared_error: 3.4517

### Inferecing

In [18]:
import mlflow
from mlflow.models import validate_serving_input

model_uri = 'runs:/ad0cff9a6d3845ac9ca8c09d7e7107cb/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Convert our test data to serving and validate le model
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


array([[4.8904753],
       [5.0173883],
       [5.524267 ],
       [5.076612 ],
       [5.2937517],
       [5.200225 ],
       [5.324072 ],
       [4.843277 ],
       [5.711568 ],
       [5.627164 ],
       [6.1908255],
       [5.4057417],
       [5.4930677],
       [5.4484887],
       [5.039768 ],
       [7.089918 ],
       [4.9057817],
       [4.639456 ],
       [7.069828 ],
       [4.7531157],
       [4.701192 ],
       [5.0614777],
       [5.872367 ],
       [6.3206644],
       [5.2239885],
       [5.516858 ],
       [6.521905 ],
       [4.686844 ],
       [5.237565 ],
       [6.5409145],
       [4.734047 ],
       [5.811737 ],
       [6.4017344],
       [5.048086 ],
       [5.5598564],
       [5.1806097],
       [6.10071  ],
       [5.778612 ],
       [5.486617 ],
       [6.589469 ],
       [5.903269 ],
       [5.1930027],
       [6.720581 ],
       [5.0198326],
       [6.2158566],
       [5.741046 ],
       [6.866602 ],
       [5.9097824],
       [4.8056016],
       [5.3147125],


## Load the model with Pyfunc

In [22]:
loaded_model = mlflow.pyfunc.load_model(model_uri)

In [23]:
# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

 1/13 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


array([[4.8904753],
       [5.0173883],
       [5.524267 ],
       [5.076612 ],
       [5.2937517],
       [5.200225 ],
       [5.324072 ],
       [4.843277 ],
       [5.711568 ],
       [5.627164 ],
       [6.1908255],
       [5.4057417],
       [5.4930677],
       [5.4484887],
       [5.039768 ],
       [7.089918 ],
       [4.9057817],
       [4.639456 ],
       [7.069828 ],
       [4.7531157],
       [4.701192 ],
       [5.0614777],
       [5.872367 ],
       [6.3206644],
       [5.2239885],
       [5.516858 ],
       [6.521905 ],
       [4.686844 ],
       [5.237565 ],
       [6.5409145],
       [4.734047 ],
       [5.811737 ],
       [6.4017344],
       [5.048086 ],
       [5.5598564],
       [5.1806097],
       [6.10071  ],
       [5.778612 ],
       [5.486617 ],
       [6.589469 ],
       [5.903269 ],
       [5.1930027],
       [6.720581 ],
       [5.0198326],
       [6.2158566],
       [5.741046 ],
       [6.866602 ],
       [5.9097824],
       [4.8056016],
       [5.3147125],


## Register the model

In [24]:
## Register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1745788380387, current_stage='None', description=None, last_updated_timestamp=1745788380387, name='wine-quality', run_id='ad0cff9a6d3845ac9ca8c09d7e7107cb', run_link=None, source='file:///workspace/complete-mlops-bootcamp/mlflow/2-DL-Project/mlruns/908217954037204112/ad0cff9a6d3845ac9ca8c09d7e7107cb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>